In [18]:
import json
from shapely.geometry import Point, Polygon, MultiPolygon
import geojson
import geopandas as gpd
import pandas as pd
import numpy as np
from time import time
import leafmap.foliumap as leafmap
import supervision as sv
from garuda.od import ConfusionMatrix
from IPython.display import Markdown

In [19]:
region = 'dhaka'
prediction_file = f"../data/predict/processed_labels/{region}/predictions_0.25.geojson"
print(f"Region: {region}")
print(f"Prediction file: {prediction_file}")

Region: dhaka
Prediction file: ../data/predict/processed_labels/dhaka/predictions_0.25.geojson


In [20]:
prediction_gdf = gpd.read_file(prediction_file) 
prediction_gdf['yolo_label'] = prediction_gdf['yolo_label'].apply(lambda x: np.array(list(map(float, x.split("_")))))
print(len(prediction_gdf))
print(prediction_gdf.crs)
prediction_gdf.head()



1253
EPSG:3857


,x,y,class_name,confidence,yolo_label,geometry
0,10052083,2632488,Zigzag,0.892156,"[2.0, 0.896215, 0.301542, 0.910111, 0.309495, ...","POLYGON ((10053294.523 2633095.647, 10053337.0..."
1,10016310,2629737,FCBK,0.373006,"[1.0, 0.17963, 0.840375, 0.182204, 0.828969, 0...","POLYGON ((10015331.049 2628696.442, 10015338.9..."
2,10068593,2637992,Zigzag,0.539420,"[2.0, 0.428197, 0.976926, 0.428861, 0.991392, ...","POLYGON ((10068373.965 2636534.139, 10068375.9..."
3,10068593,2643495,Zigzag,0.905539,"[2.0, 0.561424, 0.130893, 0.573838, 0.128923, ...","POLYGON ((10068781.304 2644624.335, 10068819.2..."
4,10068593,2643495,Zigzag,0.844645,"[2.0, 0.270389, 0.357973, 0.271333, 0.372421, ...","POLYGON ((10067891.47 2643930.042, 10067894.35..."


In [21]:
prediction_gdf.crs.to_string()
print(prediction_gdf.tail(2))

             x        y class_name  confidence  \
1251  10120876  2811351     Zigzag    0.902449   
1252  10115372  2819606     Zigzag    0.883115   

                                             yolo_label  \
1251  [2.0, 0.967177, 0.706902, 0.967753, 0.692075, ...   
1252  [2.0, 0.5007, 0.0565549, 0.501918, 0.0713807, ...   

                                               geometry  
1251  POLYGON ((10122304.813 2810718.912, 10122306.5...  
1252  POLYGON ((10115375.103 2820962.535, 10115378.8...  


In [22]:
prediction_gdf.reset_index(inplace=True, drop=True)
print(prediction_gdf.head(2))

          x        y class_name  confidence  \
0  10052083  2632488     Zigzag    0.892156   
1  10016310  2629737       FCBK    0.373006   

                                          yolo_label  \
0  [2.0, 0.896215, 0.301542, 0.910111, 0.309495, ...   
1  [1.0, 0.17963, 0.840375, 0.182204, 0.828969, 0...   

                                            geometry  
0  POLYGON ((10053294.523 2633095.647, 10053337.0...  
1  POLYGON ((10015331.049 2628696.442, 10015338.9...  


In [23]:
intersection_gdf = gpd.sjoin(prediction_gdf, prediction_gdf, predicate="intersects")[['index_right']].reset_index()
intersection_gdf = intersection_gdf[intersection_gdf['index'] < intersection_gdf['index_right']]
len(intersection_gdf)

25

In [24]:
def get_iou(row):
    left_polygon = prediction_gdf.iloc[int(row['index'])]['geometry']
    right_polygon = prediction_gdf.iloc[int(row['index_right'])]['geometry']
    left_area = left_polygon.area
    right_area = right_polygon.area
    iou = left_polygon.intersection(right_polygon).area / left_polygon.union(right_polygon).area
    return iou, left_area, right_area

intersection_gdf[['iou', 'left_area', 'right_area']] = intersection_gdf.apply(get_iou, axis=1, result_type="expand")
intersection_gdf = intersection_gdf[intersection_gdf['iou'] > 0.33]
len(intersection_gdf)
print(intersection_gdf.head(2))

     index  index_right       iou    left_area   right_area
80      80           82  0.409502  4479.786974  2782.842635
145    142          144  0.333169  1618.153862  3955.580728


In [25]:
drop_idx = intersection_gdf.apply(lambda x: x['index'] if x['left_area'] < x['right_area'] else x['index_right'], axis=1).astype(int).values
print(len(prediction_gdf))
prediction_gdf.drop(drop_idx, inplace=True)
print(len(prediction_gdf))
display(prediction_gdf.head(2))

1253
1239


,x,y,class_name,confidence,yolo_label,geometry
0,10052083,2632488,Zigzag,0.892156,"[2.0, 0.896215, 0.301542, 0.910111, 0.309495, ...","POLYGON ((10053294.523 2633095.647, 10053337.0..."
1,10016310,2629737,FCBK,0.373006,"[1.0, 0.17963, 0.840375, 0.182204, 0.828969, 0...","POLYGON ((10015331.049 2628696.442, 10015338.9..."


In [26]:
final_kilns=prediction_gdf.to_crs(epsg=4326)
display(final_kilns.head(2))

,x,y,class_name,confidence,yolo_label,geometry
0,10052083,2632488,Zigzag,0.892156,"[2.0, 0.896215, 0.301542, 0.910111, 0.309495, ...","POLYGON ((90.31028 23.00891, 90.31066 23.0087,..."
1,10016310,2629737,FCBK,0.373006,"[1.0, 0.17963, 0.840375, 0.182204, 0.828969, 0...","POLYGON ((89.96925 22.97253, 89.96932 22.97281..."


In [27]:
import os
save_handvalidation_dir = f"../hand_validation/{region}"
os.makedirs(save_handvalidation_dir, exist_ok=True)

In [28]:
len(final_kilns)

1239

In [29]:
version = "v1"  
batch_size = 500
for i in range(0, len(final_kilns), batch_size):
    end = min(i+batch_size, len(final_kilns))
    final_kilns.iloc[i:i+batch_size].to_file(f"../hand_validation/{region}/{version}_potential_kiln_{region}_{i}_{end-1}.geojson", driver='GeoJSON')

GARUDA INFO     : Created 500 records
GARUDA INFO     : Created 500 records
GARUDA INFO     : Created 239 records
